<a href="https://colab.research.google.com/github/dvschultz/ai/blob/master/NFP_Pix2PixHD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Next Frame Prediction using Pix2PixHD

By Derrick Schultz

Forked repo and tutorial based on [JC Testud’s excellent repo and Medium](https://medium.com/@jctestud/video-generation-with-pix2pix-aed5b1b69f57) article.

---


If you find this notebook useful, consider signing up for my [Patreon](https://www.patreon.com/bustbright) or [YouTube channel](https://www.youtube.com/channel/UCaZuPdmZ380SFUMKHVsv_AA/join). You can also send me a one-time payment on [Venmo](https://venmo.com/Derrick-Schultz).

## Setup

Let’s start by checking to see what GPU you’re assigned.

In [1]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-ffc4d9a8-ab15-946a-f5ab-57913e0112cb)


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Install libraries and dependencies



In [3]:
import os
if os.path.isdir("/content/drive/MyDrive/nfp-colab/pix2pixHD/"):
  %cd /content/drive/MyDrive/nfp-colab/pix2pixHD/
else:
  %cd /content/drive/MyDrive
  !mkdir nfp-colab
  %cd nfp-colab
  !git clone -b video https://github.com/dvschultz/pix2pixHD.git
  %cd pix2pixHD/

!pip install dominate

/content/drive/MyDrive/nfp-colab/pix2pixHD


## Extract frames from video

Upload a video to either Colab or Google Drive. I’d recommend Colab for speed.

In [ ]:
!python extract_frames.py -video /content/glitch-circle-white-720.mov -name glitch-circle-white_dataset -p2pdir . -width 1280 -height 736

## Train your model

### Initial training

Note: if you have a large dataset, this may timeout initially (`ValueError: __len__() should return >= 0`). Give it a minute or two and run it again.

In [ ]:
!python train_video.py --name waves-2021 --dataroot ./datasets/waves-2021/ --save_epoch_freq 5 #--continue_train

### Continue Training

In [ ]:
!python train_video.py --name waves-2021 --dataroot ./datasets/waves-2021/ --save_epoch_freq 5 --continue_train

##Generating Videos

**Options for generation process**

`--name`: Best practice is to have this match your training name

`--dataset`: Path to dataset

`--fps`: Framerate of the output video

`--how_many`: Number of frames to produce (this divided by `fps` equals the length of the video in seconds)

`--which_epoch`: By default this is `latest`. If you want to change that you can enter an integer, but it must match the number of a file in your `checkpoints` folder.

In [ ]:
!python generate_video.py --name waves-2021 --dataroot ./datasets/waves-2021/ --fps 24 --how_many 600 --which_epoch latest

### Start From

By default the model begins inference with the 50th frame in your dataset. You can change this with the `start_from` argument.

`--start_from`: path to an image (this image must have the same dimensions as the model was trained on)


In [ ]:
!python generate_video.py --name waves-2021 --dataroot ./datasets/waves-2021/ --fps 24 --how_many 600 --which_epoch latest --start_from 

### Zoom Level

You may notice that some models end up in a place of statis—after a while the frames don’t change. One way to perturb this state is with zoom_lvl. This crops in on each frame at inference to “stir” up the input.

`--zoom_lvl`: enter a pixel value (as an integer). This designates the amount of zoom per frame.

In [ ]:
!python generate_video.py --name waves-2021 --dataroot ./datasets/waves-2021/ --fps 24 --how_many 600 --which_epoch latest --zoom_lvl 10

##Experimental

In [ ]:
import os

def processFolder(folder, epoch = 10, frameCount = 240, skipCount = 1):
  files = os.listdir(folder)

  count = 0
  for f in files:
    
    if (count % skipCount == 0):
      print(f)
      if epoch == 'latest':
        command = 'python generate_video.py --name glitch_white_circle --dataroot ./datasets/glitch-circle-white_dataset/ --fps 60 --how_many %i --which_epoch latest --start_from %s/%s' % ( frameCount, folder, f)
      else:
        command = 'python generate_video.py --name glitch_white_circle --dataroot ./datasets/glitch-circle-white_dataset/ --fps 24 --how_many %i --which_epoch %i --start_from %s/%s' % ( frameCount, epoch, folder, f)
      os.system(command)
    count += 1

processFolder('./datasets/white/train_frames',1,600,3050)